In [ ]:
#DATA 
# to import data from google map - export map as KML then convert to geojson using an online converter
# each category of points will be a file
# census data from downloaded csvs off of the data table explorers
# converted to a more logical format by hand

# %pip install --force-reinstall -v "folium==0.17.0"
# %pip install geopandas
# %pip install pygris

In [ ]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import folium
import pygris as pygris
import folium.plugins as fpl
import numpy as np
import branca.colormap as cm

In [ ]:
# add parks and reserves 
parks = gpd.read_file("Vernon_County_EV_Charger_Recommendations/Parks_and_Reserves.geojson")
# add libraries
libraries = gpd.read_file("Vernon_County_EV_Charger_Recommendations/Libraries.geojson")
#add medical centers
medical_centers = gpd.read_file("Vernon_County_EV_Charger_Recommendations/Hospitals_and_Clinics.geojson")
# add high schools
city_HS = gpd.read_file("Vernon_County_EV_Charger_Recommendations/Vernon_Cty_High_Schools.geojson")
# add villages and cities w/o charges
no_charger = gpd.read_file("Vernon_County_EV_Charger_Recommendations/Villages_and_Cities_without_charging.geojson")
# mark L2 chargers
l2_charger = gpd.read_file("Vernon_County_EV_Charger_Recommendations/Existing_L2_Chargers.geojson")
# mark DC fast chargers
dc_charger = gpd.read_file("Vernon_County_EV_Charger_Recommendations/Existing_DC_Fast_Chargers.geojson")
# mark potential sites
potential_dc_charger = gpd.read_file("Vernon_County_EV_Charger_Recommendations/Potential_DC_Fast_Chargers.geojson")

#stitch the dfs together
parks['color'] = "darkgreen"
parks['icon'] = "tent"
libraries['color'] = "blue"
libraries['icon'] = "book"
medical_centers['icon'] = "hospital"
medical_centers['color'] = "darkred"
city_HS["color"] = "lightgreen"
city_HS["icon"] = "school"
l2_charger["color"] = "orange"
l2_charger["icon"] = "plug"
dc_charger["color"] = "lightred"
dc_charger["icon"] = "bolt"
no_charger["color"] = "gray"
no_charger["icon"] = "x"
potential_dc_charger["color"] = "lightblue"
potential_dc_charger["icon"] = "question"

# provided_POIs = pd.concat([parks, libraries, medical_centers, city_HS, l2_charger, dc_charger, no_charger, potential_dc_charger], axis=0)
POIs = [parks, libraries, medical_centers, city_HS, l2_charger, dc_charger, no_charger, potential_dc_charger]
names = ["Parks", "Libraries", "Medical Centers", "High Schools", "L2 Chargers", "DC Chargers", "No Charger", "Potential DC Charger"]

In [ ]:
wi_counties = pygris.counties(state="WI", cache=True)
vernon_outline = wi_counties[wi_counties['NAME'] == "Vernon"]
vernon_outline = vernon_outline.to_crs(epsg=4326)
vernon_outline

In [ ]:
#add census tract data
census_income = pd.read_csv("census_income.csv")
census_pop = pd.read_csv("census_population.csv")
census_pop

In [ ]:
tracts = pygris.tracts(state="WI", cache=True)
tracts = tracts.to_crs(epsg=4326)
tracts = tracts[tracts["COUNTYFP"] == "123"]
tracts["NAME"] = tracts["NAME"].apply(int)
tract_income = tracts.merge(census_income, how="left", left_on="NAME", right_on="Census Tract")
tract_in_pop = tract_income.merge(census_pop, how="left", on="Census Tract")
tract_in_pop

In [ ]:
#https://geopandas.org/en/stable/gallery/plotting_with_folium.html
map_vernon = folium.Map([43.44476, -90.76852], zoom_start = 10,tiles='https://tile.openstreetmap.org/{z}/{x}/{y}.png', attr='My Data Attribution')

#add markers from Google Map
x = 0
for poi in POIs:
    # Create a geometry list from the GeoDataFrame
    geo_df_list = [[point.xy[1][0], point.xy[0][0]] for point in poi.geometry]
    fg = folium.FeatureGroup(name=names[x], show="True").add_to(map_vernon)

    x = x + 1
    
    i=0
    for coordinates in geo_df_list:
        # icon = fpl.BeautifyIcon(
        #     icon=provided_POIs.iloc[i]['icon'], border_color=provided_POIs.iloc[i]['color'], text_color=provided_POIs.iloc[i]['color'], icon_shape="square", inner_icon_style='margin-top:0;'
        # )
        # Place the markers with the popup labels and data
        description = str(poi.iloc[i]['description']) if "description" in poi.iloc[i].keys() else ""
        folium.Marker(
                location=coordinates,
                popup=folium.Popup("Name: "
                + str(poi.iloc[i]['Name'])
                + "<br>"
                + "Description: "
                + str(description)),
                # icon=icon
                icon = folium.Icon(color="%s" % poi.iloc[i]['color'])
            ).add_to(fg)
        i = i + 1

#add the county line
fg = folium.FeatureGroup(name="County Line", show="True").add_to(map_vernon)
#add the county outline
#https://geopandas.org/en/stable/gallery/polygon_plotting_with_folium.html
for _, r in vernon_outline.iterrows():
    sim_geo = gpd.GeoSeries(r["geometry"]).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j, style_function=lambda x: {"fillColor": "transparent"})
    geo_j.add_to(fg)

values = tract_in_pop["Median Household income (dollars)"].unique()
cmap = cm.linear.RdPu_05.scale(min(values), max(values))

#add the census demographics
fg = folium.FeatureGroup(name="Census Demographics", show="True").add_to(map_vernon)
for _, r in tract_in_pop.iterrows():
  inc = r["Median Household income (dollars)"]
  sim_geo = gpd.GeoSeries(r["geometry"]).simplify(tolerance=0.001)
  geo_j = sim_geo.to_json()
  geo_j = folium.GeoJson(data=geo_j, style_function=lambda x, inc=inc: {
    'fillColor': cmap(inc)
  })
  folium.Popup("Inc: " + str(r["Median Household income (dollars)"]) + " Pop: " + str(r["Population"]) ).add_to(geo_j)
  geo_j.add_to(fg)

#add layer controls
folium.LayerControl().add_to(map_vernon)
map_vernon

In [78]:
#work out the icons?
#add jobs, demographics, housing?
#FAILED NLCD LANDUSE EXPERIMENT

# vernon_outline.index = ["Vernon"]
# lulc = gh.nlcd_bygeom(vernon_outline, 100, years={"cover": [2021]})
# fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(9, 4))

# cmap, norm, levels = gh.plot.cover_legends()
# cover = lulc["Vernon"].cover_2021
# cover
# cover.where(cover < 127).plot(ax=ax1, cmap=cmap, levels=levels, cbar_kwargs={"ticks": levels[:-1]})
# ax1.set_title("Land Use/Land Cover 2019")
# ax1.set_axis_off()
# cover_df = cover.to_dataframe().reset_index()
# cover_df["color"] = cover_df["cover_2021"].apply(cmap)
# geo_cover_df = gpd.GeoDataFrame(
#     cover_df, geometry=gpd.points_from_xy(cover_df.y, cover_df.x), crs="EPSG:4326"
# )
# geo_cover_df
# geo_cover_df = geo_cover_df.to_crs(espg=3395)
# buffer = geo_cover_df.buffer(30, cap_style=3)
# for _, r in cover_df.iterrows():
#     sim_geo = gpd.GeoSeries(r["geometry"]).simplify(tolerance=0.001)
#     geo_j = sim_geo.to_json()
#     geo_j = folium.GeoJson(data=geo_j, style_function=lambda x: {"fillColor": "transparent"})
#     geo_j.add_to(map_vernon)

In [79]:
map_vernon.save("VernonCO_chargerMap.html")